## Last time: CNN
* Trained model
* [Geomtric deep learning](http://geometricdeeplearning.com/)

## Non-image CNN: Sequence motif
* [Position weight matrix](https://en.wikipedia.org/wiki/Position_weight_matrix)

![](img/pwm.png)

![](http://jaspar.genereg.net/static/logos/svg/MA0149.1.svg)

In [1]:
import numpy as np

seq_length = 40
num_train = 1000
num_val = 100

# PFM from JASPAR
motif = np.array([[   0,   2, 104, 104,   1,   2, 103, 102,   0,   0,  99, 105,   0,   0, 100, 102,   5,   3],
                  [   0,   0,   0,   0,   0,   0,   0,   0,   0,   2,   4,   0,   0,   2,   3,   0,   0,   3],
                  [ 105, 103,   1,   1, 104, 102,   2,   3, 104, 103,   2,   0, 105, 103,   0,   2,  97,  97],
                  [   0,   0,   0,   0,   0,   1,   0,   0,   1,   0,   0,   0,   0,   0,   2,   1,   3,   2]])

In [2]:
def datagen(seq_length, num_sample, motif):
    from tensorflow.keras.utils import to_categorical
    
    freq = np.hstack([np.ones((4,(seq_length-motif.shape[1])//2)), 
                      motif,
                      np.ones((4,(seq_length-motif.shape[1])//2))])

    #normalize to PWM and generate positive samples
    pos = np.array([np.random.choice(['A', 'C', 'G', 'T'], num_sample, p=freq[:,i]/sum(freq[:,i])) 
                    for i in range(seq_length)]).transpose()
    [''.join(x) for x in pos[1:10,:]]
    
    neg = np.array([np.random.choice(['A', 'C', 'G', 'T'], num_sample, p=np.array([1,1,1,1])/4.0)
                for i in range(seq_length)]).transpose()

    [''.join(x) for x in neg[1:10,:]]
    
    pos_tensor = np.zeros(list(pos.shape) + [4])
    neg_tensor = np.zeros(list(neg.shape) + [4])

    base_dict = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

    #naive one-hot encoding
    for row in range(num_sample):
        for col in range(seq_length):
            pos_tensor[row,col,base_dict[pos[row,col]]] = 1
            neg_tensor[row,col,base_dict[neg[row,col]]] = 1

    X = np.vstack((pos_tensor, neg_tensor))
    y = to_categorical(np.concatenate((np.ones(num_sample), np.zeros(num_sample))), 2)
    return X, y

In [3]:
def dataset(seq_length, num_train, num_val, motif):
    X_train, y_train = datagen(seq_length=seq_length,
                               num_sample=num_train,
                               motif=motif)
    X_val, y_val = datagen(seq_length=seq_length,
                           num_sample=num_val,
                           motif=motif)
    return (X_train, y_train), (X_val, y_val)

(X_train, y_train), (X_val, y_val) = dataset(seq_length, num_train, num_val, motif)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout, MaxPooling1D
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import SGD

model = Sequential()
model.add(Conv1D(filters=1, 
                 kernel_size=17,
                 padding='same',
                 input_shape=(seq_length, 4),
                 activation='relu'))

model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 40, 1)             69        
_________________________________________________________________
flatten_1 (Flatten)          (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 82        
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________


In [13]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, validation_split=0.3, epochs=10, shuffle=True)  # starts training

Train on 14000 samples, validate on 6000 samples
Epoch 1/10
 6304/14000 [============>.................] - ETA: 3s - loss: 0.0709 - acc: 0.9683

KeyboardInterrupt: 

In [11]:
score = model.evaluate(X_val, y_val, verbose=0)
print(score[1])

1.0


In [7]:
convlayer = model.layers[0]
weights = convlayer.get_weights()[0].squeeze()
print('Convolution parameter shape: {}'.format(weights.shape))

Convolution parameter shape: (18, 4)


In [8]:
num2seq = ['A','C','G','T']

''.join([num2seq[np.argmax(weights[i,:])] for i in range(weights.shape[0])])

'TGGAAGGAAGGAATGTTC'

# Recurrent Neural Networks
**by: Santiago Hincapie**